In [49]:
import pandas as pd, json, numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
%matplotlib inline

Load airports of each country

In [50]:
L=json.loads(file('../json/L.json','r').read())
M=json.loads(file('../json/M.json','r').read())
N=json.loads(file('../json/N.json','r').read())

In [51]:
import requests

In [52]:
AP={}
for c in M:
    if c not in AP:AP[c]={}
    for i in range(len(L[c])):
        AP[c][N[c][i]]=L[c][i]

record schedules for 2 weeks, then augment count with weekly flight numbers.
seasonal and seasonal charter will count as once per week for 3 months, so 12/52 per week. TGM separate, since its history is in the past.

parse Departures

In [53]:
baseurl='https://www.airportia.com/'
import requests, urllib2

In [54]:
def urlgetter(url):
    s = requests.Session()
    cookiesopen = s.get(url)
    cookies=str(s.cookies)
    fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
    #push token
    opener = urllib2.build_opener()
    for k in fcookies:
        opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
    #read html
    return s.get(url).content

good dates

In [55]:
SD={}
SC=json.loads(file('../json/SC2.json','r').read())

In [56]:
#pop out last - if applicable
try: SD.pop(c)
except: pass
for h in range(len(AP.keys())):
    c=AP.keys()[h]
    #country not parsed yet
    if c in SC:
        if c not in SD:
            SD[c]=[]
            print h,c
            airportialinks=AP[c]
            sch={}
            #all airports of country, where there is traffic
            for i in airportialinks:
                if i in SC[c]:
                    print i,
                    if i not in sch:sch[i]={}
                    url=baseurl+airportialinks[i]
                    m=urlgetter(url)
                    for d in range (3,17):
                        #date not parsed yet
                        if d not in sch[i]:
                            url=baseurl+airportialinks[i]+'departures/201704'+str(d)
                            m=urlgetter(url)
                            soup = BeautifulSoup(m, "lxml")
                            #if there are flights at all
                            if len(soup.findAll('table'))>0:
                                sch[i][d]=pd.read_html(m)[0] 
                            else: print '--W-',d,
            SD[c]=sch
            print 

0 Canada
YUY YUX --W- 8 --W- 15 YUL XBE --W- 9 --W- 16 YUB YUD YRT YUT LAK ZFN --W- 9 --W- 16 YBX --W- 15 YBR YBT YBV --W- 15 YBK YBL YBB --W- 11 YBC YLL --W- 8 --W- 9 --W- 13 --W- 14 --W- 15 --W- 16 YBF --W- 3 --W- 4 --W- 5 --W- 6 --W- 7 --W- 8 --W- 9 --W- 10 --W- 11 --W- 12 --W- 13 --W- 14 --W- 15 --W- 16 YBG ZGR --W- 3 --W- 4 --W- 5 --W- 6 --W- 7 --W- 8 --W- 9 --W- 10 --W- 11 --W- 12 --W- 13 --W- 14 --W- 15 --W- 16 XKS --W- 9 --W- 16 ZGI AKV YIF --W- 8 --W- 15 YIO YIK YIV XTL ZMT --W- 8 --W- 15 YFA YFB YFC YFO YFH --W- 8 --W- 9 --W- 15 --W- 16 YFJ --W- 8 --W- 15 YFS YPR YPQ --W- 3 --W- 4 --W- 5 --W- 6 --W- 7 --W- 8 --W- 9 --W- 10 --W- 11 --W- 12 --W- 13 --W- 14 --W- 15 --W- 16 YPX YPY --W- 4 --W- 6 --W- 8 --W- 9 --W- 11 --W- 13 --W- 15 --W- 16 YPB --W- 3 --W- 14 --W- 16 YPC --W- 3 --W- 6 --W- 8 --W- 9 --W- 10 --W- 13 --W- 15 --W- 16 YPA --W- 3 --W- 4 --W- 5 --W- 6 --W- 7 --W- 8 --W- 9 --W- 10 --W- 11 --W- 12 --W- 13 --W- 14 --W- 15 --W- 16 YPJ YPH YPN --W- 5 --W- 8 --W- 9 --W- 12 --

Save

In [57]:
dbpath='E:/Dropbox/Public/datarepo/aviation/' #large file db path
file(dbpath+"json/SD_dest.json",'w').write(repr(SD))

In [58]:
I3=json.loads(file('../json/I3.json','r').read())

In [59]:
MDF=pd.DataFrame()

In [60]:
for c in SD:
    sch=SD[c]
    mdf=pd.DataFrame()
    for i in sch:
        for d in sch[i]:
            df=sch[i][d].drop(sch[i][d].columns[3:],axis=1).drop(sch[i][d].columns[0],axis=1)
            df['From']=i
            df['Date']=d
            mdf=pd.concat([mdf,df])
    if len(sch)>0:
        mdf['City']=[i[:i.rfind(' ')] for i in mdf['To']]
        mdf['Airport']=[i[i.rfind(' ')+1:] for i in mdf['To']]
        cpath=I3[c].lower()
        file('../countries/'+cpath+"/json/mdf_dest.json",'w').write(json.dumps(mdf.reset_index().to_json()))
        MDF=pd.concat([MDF,mdf])
        print c,

Canada Turkmenistan Saint Helena Lithuania Cambodia Ethiopia Aruba Perú Argentina Bolivia Cameroon Burkina Faso Ghana Saudi Arabia Martinique Jordan Cape Verde United States Minor Outlying Islands Cocos (Keeling) Islands Slovenia Guatemala Kuwait Wallis And Futuna Dominica Liberia Maldives Pakistan Oman Tanzania Saint Kitts And Nevis Albania Gabon Saint Pierre and Miquelon Monaco Bahamas New Zealand Yemen Jamaica Greenland Samoa United Arab Emirates Guam Uruguay India Azerbaijan Lesotho Saint Vincent and the Grenadines Kenya South Korea Tajikistan Turkey Afghanistan Northern Mariana Islands Mauritania Solomon Islands Turks and Caicos Islands Saint Lucia Hungary Mongolia France Bermuda Slovakia Somalia Laos Nauru Norway Malawi Cook Islands Benin Libya Cuba Montenegro Falkland Islands (Malvinas) Togo China Armenia Dominican Republic Ukraine Bahrain Tonga Cayman Islands Western Sahara Finland Central African Republic Mauritius Sweden Vietnam Antigua And Barbuda Mali Russia Bulgaria United

In [61]:
dbpath='E:/Dropbox/Public/datarepo/aviation/' #large file db path
MDF.reset_index().to_json(dbpath+'json/MDF_dest.json')